In [1]:
import scipy
import numpy as np

In [9]:
from sklearn.cluster import MiniBatchKMeans
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import OneSidedSelection
from imblearn.pipeline import Pipeline
from sklearn.model_selection import KFold

In [2]:
data=scipy.io.loadmat('DBPB.mat')#读入由matlab处理好的样本特征向量
data_test=scipy.io.loadmat('DBPB_test.mat')

In [3]:
data['DBPB1'][0].shape#每个样本94个特征

(94,)

In [5]:
data['DBPB1']

array([[0.00172604, 0.00379138, 0.00296589, ..., 0.00116799, 0.00091695,
        0.00525093],
       [0.00352267, 0.00097731, 0.00296935, ..., 0.00847652, 0.00462204,
        0.00282324],
       [0.00339284, 0.00166496, 0.00277194, ..., 0.00288942, 0.00499452,
        0.00355836],
       ...,
       [0.00799746, 0.00392692, 0.00227704, ..., 0.00519247, 0.00426208,
        0.00183095],
       [0.00149238, 0.00276466, 0.00239477, ..., 0.00100603, 0.00415174,
        0.00158   ],
       [0.00478357, 0.00589117, 0.006869  , ..., 0.00381122, 0.01051007,
        0.01011765]])

In [4]:
data['DBPB1'].shape#第一类样本数量为9279

(9279, 94)

### 先将数据读入到矩阵中便于处理

In [6]:
d_1=np.array(data['DBPB1'])
d_2=np.array(data['DBPB2'])
d_3=np.array(data['DBPB3'])
d_4=np.array(data['DBPB4'])
d_5=np.array(data['DBPB5'])
d_6=np.array(data['DBPB6'])
d_7=np.array(data['DBPB7'])
d_8=np.array(data['DBPB8'])
d_9=np.array(data['DBPB9'])
d_10=np.array(data['DBPB10'])
d_11=np.array(data['DBPB11'])

dt_1=np.array(data_test['DBPB1'])
dt_2=np.array(data_test['DBPB2'])
dt_3=np.array(data_test['DBPB3'])
dt_4=np.array(data_test['DBPB4'])
dt_5=np.array(data_test['DBPB5'])
dt_6=np.array(data_test['DBPB6'])
dt_7=np.array(data_test['DBPB7'])
dt_8=np.array(data_test['DBPB8'])
dt_9=np.array(data_test['DBPB9'])
dt_10=np.array(data_test['DBPB10'])
dt_11=np.array(data_test['DBPB11'])


### 接下来进行数据平衡

#### 整合数据

In [20]:
X_train=np.concatenate((d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11),axis=0)
X_test=np.concatenate((dt_1,dt_2,dt_3,dt_4,dt_5,dt_6,dt_7,dt_8,dt_9,dt_10,dt_11),axis=0)
print(X_train.shape)
print(X_test.shape)

(12983, 94)
(5662, 94)


In [21]:
for i in range(1,12):
    exec(f'y_{i}=np.full(d_{i}.shape[0],i)');
    exec(f'yt_{i}=np.full(dt_{i}.shape[0],i)')
y_dir=[y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11]
yt_dir=[yt_1,yt_2,yt_3,yt_4,yt_5,yt_6,yt_7,yt_8,yt_9,yt_10,yt_11]
y=np.concatenate((y_dir),axis=0)
yt=np.concatenate((yt_dir),axis=0)
print(y.shape)
print(yt.shape)

(12983,)
(5662,)


In [22]:
def ratio_multiplier(y):
    from collections import Counter

    multiplier = {1: 0.334}
    target_stats = Counter(y)
    for key, value in target_stats.items():
        if key in multiplier:
            target_stats[key] = int(value * multiplier[key])
    return target_stats

ratio_multiplier(y)

Counter({1: 3099,
         2: 710,
         3: 600,
         5: 561,
         4: 454,
         10: 454,
         7: 360,
         6: 251,
         9: 153,
         8: 88,
         11: 73})

In [27]:
under=RandomUnderSampler(sampling_strategy=ratio_multiplier,random_state=1)
x_under_sampled,y_under_sampled=under.fit_resample(X_train,y)
over=RandomOverSampler(random_state=1)
x_resampled,y_resampled=over.fit_resample(x_under_sampled,y_under_sampled)
for i in range(1,12):
    print(sum(y_resampled==i))

3099
3099
3099
3099
3099
3099
3099
3099
3099
3099
3099


In [28]:
X_train=x_resampled
Y_train=np.zeros((y_resampled.shape[0],4))
Y_test=np.zeros((yt.shape[0],4))
for i in range(y_resampled.shape[0]):
    if y_resampled[i]==1:
        Y_train[i]=np.array([1,0,0,0])#a
    elif y_resampled[i]==2:
        Y_train[i]=np.array([0,1,0,0])#c
    elif y_resampled[i]==3:
        Y_train[i]=np.array([0,0,1,0])#m
    elif y_resampled[i]==4:
        Y_train[i]=np.array([0,0,0,1])#s
    elif y_resampled[i]==5:
        Y_train[i]=np.array([1,1,0,0])#ac
    elif y_resampled[i]==6:
        Y_train[i]=np.array([1,0,1,0])#am
    elif y_resampled[i]==7:
        Y_train[i]=np.array([1,0,0,1])#as
    elif y_resampled[i]==8:
        Y_train[i]=np.array([0,1,1,0])#cm
    elif y_resampled[i]==9:
        Y_train[i]=np.array([1,1,1,0])#acm
    elif y_resampled[i]==10:
        Y_train[i]=np.array([1,1,0,1])#acs
    elif y_resampled[i]==11:
        Y_train[i]=np.array([1,1,1,1])#acms

for i in range(yt.shape[0]):
    if yt[i]==1:
        Y_test[i]=np.array([1,0,0,0])#a
    elif yt[i]==2:
        Y_test[i]=np.array([0,1,0,0])#c
    elif yt[i]==3:
        Y_test[i]=np.array([0,0,1,0])#m
    elif yt[i]==4:
        Y_test[i]=np.array([0,0,0,1])#s
    elif yt[i]==5:
        Y_test[i]=np.array([1,1,0,0])#ac
    elif yt[i]==6:
        Y_test[i]=np.array([1,0,1,0])#am
    elif yt[i]==7:
        Y_test[i]=np.array([1,0,0,1])#as
    elif yt[i]==8:
        Y_test[i]=np.array([0,1,1,0])#cm
    elif yt[i]==9:
        Y_test[i]=np.array([1,1,1,0])#acm
    elif yt[i]==10:
        Y_test[i]=np.array([1,1,0,1])#acs
    elif yt[i]==11:
        Y_test[i]=np.array([1,1,1,1])#acms

In [37]:
print(f"X_train:{X_train.shape[0]}")
print(f"Y_train:{Y_train.shape[0]}")
print(f"X_test:{X_test.shape[0]}")
print(f"Y_test:{Y_test.shape[0]}")

X_train:34089
Y_train:34089
X_test:5662
Y_test:5662


### 至此重采样完成，接下来开始进行分类模型构建

In [38]:
def get_knn(x,X_train,k,sign):
    diffMat=np.tile(x,(X_train.shape[0],1))-X_train
    distance=((diffMat**2).sum(axis=1))**0.5
    if sign==1:
        ans=distance.argsort()[0:k]
    else:
        ans=distance.argsort()[1:k+1]
    return ans

In [39]:
#mlknn
def mlknn_train(X_train,Y_train,k):
    s=1
    #计算训练样本是第i类的概率
    p_1=np.zeros(4)
    p_0=np.zeros(4)
    for i in range(0,4):
        p_1[i]=(s+sum(Y_train[:,i]==1))/(s*2+Y_train.shape[0])
        p_0[i]=1-p_1[i]
    print(f"p_1:{p_1}")
    #计算某样本是第i类的条件下，它的K近邻中有几个是第i类的条件概率
    #c_1记录每个样本的K近邻中有count个属于这一类的情况下该样本也属于这一类的个数
    c_1=np.zeros((4,k+1))
    c_0=np.zeros((4,k+1))
    #p_1_n记录样本属于第i类的情况下，其K近邻中有几个是第i类的条件概率
    p_1_n=np.zeros((4,k+1))
    p_0_n=np.zeros((4,k+1))
    N_x=np.zeros((X_train.shape[0],k,4))
    for j in range(0,X_train.shape[0]):
        N_x[j]=Y_train[get_knn(X_train[j],X_train,k,0)] #获得每个训练样本x的K近邻样本的标签
    for i in range(0,4):
        for j in range(0,Y_train.shape[0]):
            count=sum(N_x[j][:,i]==1)              #统计第j个训练样本的K近邻样本中属于第i类的数量
            if(Y_train[j][i]==1):
                c_1[i][count]+=1
            else:
                c_0[i][count]+=1
        #求出样本是第i类的条件下，K近邻中有j个是第i类的条件概率
        #求出样本不是第i类的条件下，K近邻中有j个是第i类的条件概率
        for j in range(0,k+1):
            p_1_n[i][j]=(s+c_1[i][j])/(s*(k+1)+sum(c_1[i]))
            p_0_n[i][j]=(s+c_0[i][j])/(s*(k+1)+sum(c_0[i]))
    return p_1,p_0,p_1_n,p_0_n

In [40]:
def mlknn_predecit(X_test,X_train,Y_train,k,p_1,p_0,p_1_n,p_0_n):
    #获取待预测样本的K近邻样本的标签
    N_x=np.zeros((X_test.shape[0],k,4))
    Y_pre=np.zeros((X_test.shape[0],4))
    for j in range(0,X_test.shape[0]):
        N_x[j]=Y_train[get_knn(X_test[j],X_train,k,1)]
    for i in range(0,4):
        for j in range(0,X_test.shape[0]):
            count=sum(N_x[j][:,i]==1)              #统计第j个训练样本的K近邻样本中属于第i类的数量
            Y_pre[j][i]=(p_1[i]*p_1_n[i][count]>p_0[i]*p_0_n[i][count])
    return Y_pre

In [41]:
#使用训练集进行K折交叉验证并返回相关评价指标
def mlknn_KfoldCV(X,Y,k,kfold):
    acc=0
    kf=KFold(n_splits=kfold,shuffle=True)
    for train_index,test_index in kf.split(X):
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=Y[train_index],Y[test_index]
        p_1,p_0,p_1_n,p_0_n=mlknn_train(x_train,y_train,k)
        y_pre=mlknn_predecit(x_test,x_train,y_train,k,p_1,p_0,p_1_n,p_0_n)
        acc_count=0
        for i in range(0,y_pre.shape[0]):
            if((y_pre[i]==y_test[i]).all()):
                acc_count+=1
        acc+=(acc_count/x_test.shape[0])
        print(acc_count/x_test.shape[0])
    acc/=kfold
    return acc

In [42]:
#使用测试集进行测试并返回相关评价指标
def mlknn_test(X_test,Y_test,X_train,Y_train,k):
    p_1,p_0,p_1_n,p_0_n=mlknn_train(X_train,Y_train,k)
    Y_pre=mlknn_predecit(X_test,X_train,Y_train,k,p_1,p_0,p_1_n,p_0_n)
    print("finish training,start predict")
    acc_count=0
    for i in range(0,Y_pre.shape[0]):
            if((Y_pre[i]==Y_test[i]).all()):
                acc_count+=1
    acc=(acc_count/X_test.shape[0])
    print(f"acc_count={acc_count},total_count={X_test.shape[0]}")
    return acc

In [ ]:
acc_test=mlknn_test(X_test,Y_test,X_train,Y_train,11)
print(f"Accuracy={acc_test}")

p_1:[0.63635564 0.54545188 0.45454812 0.36364436]
finish training,start predict
acc_count=544,total_count=5662
Accuracy=0.0960791239844578


In [32]:
acc=mlknn_KfoldCV(X_train,Y_train,5,5)

p_1:[0.86041586 0.15768194 0.09087409 0.10444744]
start type 1
finish type 1
start type 2
finish type 2
start type 3
finish type 3
start type 4
finish type 4
0.7173661917597227
p_1:[0.85666153 0.15777821 0.08788987 0.10348479]
start type 1
finish type 1
start type 2
finish type 2
start type 3
finish type 3
start type 4
finish type 4
0.7115902964959568
p_1:[0.85791298 0.15941471 0.08933385 0.103196  ]
start type 1
finish type 1
start type 2
finish type 2
start type 3
finish type 3
start type 4
finish type 4
0.7165960723912206
p_1:[0.85600154 0.15689672 0.09105785 0.10414862]
start type 1
finish type 1
start type 2
finish type 2
start type 3
finish type 3
start type 4
finish type 4
0.7172573189522342
p_1:[0.85542401 0.15381654 0.08990278 0.10154972]
start type 1
finish type 1
start type 2
finish type 2
start type 3
finish type 3
start type 4
finish type 4
0.7091679506933745


In [107]:
pre=mlknn_predecit(X_train,X_train,Y_train,5,p_1,p_0,p_1_n,p_0_n)

In [145]:
acc_c=0
for i in range(0,pre.shape[0]):
    if((pre[i]==Y_train[i]).all()):
        acc_c+=1

In [146]:
acc_c

9279

In [149]:
acc=acc_c/X_train.shape[0]

In [150]:
acc

0.7147038434876377